## NBA API

### Description
- Ingest data via nba_api

### Import Packaages

In [ ]:
## import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import nba_api
from nba_api.stats.endpoints import commonplayerinfo
from nba_api.stats.static import players
from nba_api.stats.library.parameters import SeasonAll
from nba_api.stats.static import teams as static_teams
from nba_api.stats.static import players as static_players
from nba_api.stats.endpoints import leaguegamefinder
import time
import requests
import json
import seaborn as sns
import sys

Matplotlib is building the font cache; this may take a moment.


### Test pull to grab recent ATL games

In [5]:
from nba_api.stats.endpoints import leaguegamefinder

atl = 1610612737  # Atlanta Hawks team_id
games = leaguegamefinder.LeagueGameFinder(team_id_nullable=atl).get_data_frames()[0]
games.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22025,1610612737,ATL,Atlanta Hawks,0022500223,2025-11-12,ATL @ SAC,W,240,133,...,0.818,7,38,45,42,14.0,6,19,20,33.0
1,22025,1610612737,ATL,Atlanta Hawks,0022500206,2025-11-10,ATL @ LAC,W,239,105,...,0.800,7,34,41,30,7.0,8,11,22,3.0
2,22025,1610612737,ATL,Atlanta Hawks,0022500185,2025-11-08,ATL vs. LAL,W,240,122,...,0.909,7,30,37,37,13.0,6,11,14,20.0
3,22025,1610612737,ATL,Atlanta Hawks,0022500030,2025-11-07,ATL vs. TOR,L,241,97,...,0.957,19,35,54,24,8.0,7,15,19,-12.0
4,22025,1610612737,ATL,Atlanta Hawks,0022500166,2025-11-04,ATL vs. ORL,W,240,127,...,0.810,6,28,34,23,11.0,6,18,24,15.0


In [6]:
games.columns

Index(['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID',
       'GAME_DATE', 'MATCHUP', 'WL', 'MIN', 'PTS', 'FGM', 'FGA', 'FG_PCT',
       'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB',
       'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PLUS_MINUS'],
      dtype='object')

### Lets do some broader things to get data
- first use the static databases to get team and player IDs (thesse can be dim tables)
- then lets flush out some broader functionality with usefful endpoints

#### Grab team IDs

In [8]:
# All teams (list[dict])
teams_list = static_teams.get_teams()
teams_df = pd.DataFrame(teams_list).sort_values("full_name")
teams_df.head()

,id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Georgia,1949
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946
14,1610612751,Brooklyn Nets,BKN,Nets,Brooklyn,New York,1976
29,1610612766,Charlotte Hornets,CHA,Hornets,Charlotte,North Carolina,1988
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966


#### Create some helper dictionaries 

In [9]:
# Helper maps
TEAM_BY_ABBR = {t["abbreviation"]: t for t in teams_list}
TEAM_BY_ID   = {t["id"]: t for t in teams_list}

# Example lookups
TEAM_BY_ABBR["ATL"]["id"]         # -> 1610612737
TEAM_BY_ID[1610612737]["full_name"]  # -> "Atlanta Hawks"


'Atlanta Hawks'

#### Grab Player IDs

In [14]:
# All players (current + historical)
players_list = static_players.get_players()
players_df = pd.DataFrame(players_list)
players_df.head()

,id,full_name,first_name,last_name,is_active
0,76001,Alaa Abdelnaby,Alaa,Abdelnaby,False
1,76002,Zaid Abdul-Aziz,Zaid,Abdul-Aziz,False
2,76003,Kareem Abdul-Jabbar,Kareem,Abdul-Jabbar,False
3,51,Mahmoud Abdul-Rauf,Mahmoud,Abdul-Rauf,False
4,1505,Tariq Abdul-Wahad,Tariq,Abdul-Wahad,False


In [15]:
# Find by name (returns list; pick the first if unique)
static_players.find_players_by_full_name("LeBron James")

[{'id': 2544,
  'full_name': 'LeBron James',
  'first_name': 'LeBron',
  'last_name': 'James',
  'is_active': True}]